In [76]:
# import neccessary libraries
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import re

In [ ]:
# load data
text_data = pd.read_csv()

In [8]:
!pwd

/content


In [9]:
%cd /content/drive/My\ Drive/

[Errno 2] No such file or directory: '/content/drive/My Drive/'
/content


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [18]:
# load data into pandas
text_data = pd.read_csv('./Sentiment.csv')
text_data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [20]:
# the data contains irrelevant fields, we create new dataframe to hold our text and sentiment values
new_text_df = text_data.loc[:,['text', 'sentiment']]

In [21]:
print(new_text_df.sentiment)

0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [77]:
# convert all text to lowercase and we train on only the positive and negative sentiment 
new_text_df = new_text_df[new_text_df.sentiment != 'Neutral']
new_text_df['text'] = new_text_df['text'].str.lower() 
new_text_df['text'] = new_text_df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [47]:
new_text_df.head()

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [78]:
# we need to also remove all the rts for each rows
for idx, row in new_text_df.iterrows():
  row[0] = row[0].replace('rt','')

print(new_text_df[new_text_df['sentiment'] == 'Positive'].size)
print('=====')
print(new_text_df[new_text_df['sentiment'] == 'Negative'].size)

4472
=====
16986


In [79]:
# We have more negative sentiment than than the positive. Now we need to define a vocabulary that can be used as a tokenizer to help convert our text into vectors.
tokenizer = Tokenizer(num_words=1500, split=' ')
tokenizer.fit_on_texts(new_text_df['text'].values)
# we can save all the text in a variable X
X = tokenizer.texts_to_sequences(new_text_df['text'].values)
# we use padding to bring all text values to a commen length
X = pad_sequences(X)

To define our model architecture, we will use embed_layer_dim, LSTM_out. Then we will set our output activation function as softmax for cross categorical entropy which is the ideal right activation function for our network.

## Model Building

In [80]:
embed_dim = 128
lstm_out = 196
num_words = 1500
model = Sequential()
model.add(Embedding(1500,embed_dim, input_length = 28))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation ='softmax'))

In [81]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [82]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           192000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 447,194
Trainable params: 447,194
Non-trainable params: 0
_________________________________________________________________


In [83]:
# We now encode our sentiment so it can be fitted to our modelm
Y = pd.get_dummies(new_text_df['sentiment']).values
#Y = Le.fit_transform(new_text_df['sentiment']).values

In [84]:
print(X_train.shape, Y_train.shape)
print('=========================')
print(X_test.shape, Y_test.shape)

(9119, 28) (9119, 2)
(1610, 28) (1610, 2)


In [85]:
# split our train data 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state =42)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=32)

Epoch 1/10
285/285 [==============================] - 50s 165ms/step - loss: 0.4821 - accuracy: 0.8050 - val_loss: 0.3521 - val_accuracy: 0.8422
Epoch 2/10
285/285 [==============================] - 46s 162ms/step - loss: 0.3057 - accuracy: 0.8752 - val_loss: 0.3485 - val_accuracy: 0.8571
Epoch 3/10
285/285 [==============================] - 46s 162ms/step - loss: 0.2683 - accuracy: 0.8911 - val_loss: 0.3672 - val_accuracy: 0.8516
Epoch 4/10
285/285 [==============================] - 46s 163ms/step - loss: 0.2381 - accuracy: 0.9025 - val_loss: 0.3789 - val_accuracy: 0.8509
Epoch 5/10
285/285 [==============================] - 46s 162ms/step - loss: 0.2160 - accuracy: 0.9138 - val_loss: 0.4183 - val_accuracy: 0.8503
Epoch 6/10
285/285 [==============================] - 46s 163ms/step - loss: 0.2004 - accuracy: 0.9170 - val_loss: 0.4255 - val_accuracy: 0.8416
Epoch 7/10
285/285 [==============================] - 47s 163ms/step - loss: 0.1781 - accuracy: 0.9281 - val_loss: 0.4629 - val_ac

In [86]:
model.evaluate(X_test, Y_test)

51/51 [==============================] - 1s 21ms/step - loss: 0.5838 - accuracy: 0.8311


[0.5837957262992859, 0.8310558795928955]

Our model gave 0.83% accuracy which is good but can still be further improved.Now lets compare our predictions with our actual labels.

In [87]:
print("Prediction:", model.predict_classes(X_test[5:10]))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Prediction: [1 0 0 0 0]


In [90]:
print("Actual: \n", Y_test[5:10])

Actual: 
 [[0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


The result shows our model is doing well having classfied 4 predictions correctly with one misslassification. We should also not forget we had more negative sentiment than the positive which clearly shows why the model is performing better at classfying negative sentiment accurately